In [2]:
import csv
import tensorflow as tf
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime

### Train Batch Pre-Processing

In [3]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"

with open('Interviewdata_Question.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    questions = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(questions))
with open('Interviewdata_Answer.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    answers = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(answers))
with open('Interviewdata_label.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    labels = np.array(list(itertools.chain(*[x for x in reader]))).astype(np.int32)
    print(len(labels)) 

408
408
408


In [4]:
import pickle
with open ('words.txt', 'rb') as fp:
    wordlist = pickle.load(fp)

In [5]:
with open('embeddings.txt','rb') as fp:
    embeddings = pickle.load(fp)

In [6]:
wordlist.append('UNKNOWN_TOKEN')

In [7]:
len(wordlist)

1720

In [8]:
embeddings = np.vstack([embeddings, np.zeros(200)])

In [9]:
len(embeddings)

1720

In [10]:
word_to_index = dict([(w,i) for i,w in enumerate(wordlist)])

In [11]:
len(word_to_index)

1720

In [12]:
for i, sent in enumerate(questions):
    questions[i] = [w if w in word_to_index else unknown_token for w in sent]

In [13]:
for i, sent in enumerate(answers):
    answers[i] = [w if w in word_to_index else unknown_token for w in sent]

In [14]:
import numpy as np
X_question = np.asarray([[word_to_index[w] for w in sent] for sent in questions])
y_answer = np.asarray([[word_to_index[w] for w in sent] for sent in answers])

In [15]:
X_question[:10]

array([ [5, 8, 184, 0, 72, 2, 41, 447, 136, 122, 878, 136, 122, 258, 184, 1111, 389, 362, 1228, 7],
       [5, 8, 184, 0, 72, 2, 19, 47, 1570, 3, 486, 216, 1052, 6, 1322, 3, 370, 6, 1684, 60, 26, 60, 104, 739, 6, 0, 47, 1, 5, 34, 186, 739, 6, 47, 213, 96, 15, 584, 1],
       [12, 2, 620, 82], [12, 2, 331, 817],
       [615, 352, 1124, 136, 122, 5, 33, 49, 55, 21, 156, 547, 150, 615, 11, 10, 145, 1191, 186, 12, 20, 15, 446, 7],
       [27, 1308, 0, 72, 2, 81, 41, 447, 136, 122, 541, 11, 82, 1688, 10, 136, 122, 541, 11, 728, 815, 10, 136, 122, 12, 20, 15, 446, 4, 41, 72, 7],
       [34, 17, 2, 0, 1303, 544, 17, 6, 82, 17],
       [1, 55, 0, 72, 2, 81, 41, 183, 1140, 183, 11, 10, 54, 183, 1045, 54, 183, 1044, 54, 1298, 1, 1060, 54, 12, 20, 446, 4, 1197, 7],
       [38, 440, 13, 597, 33, 24, 2, 763, 19, 148, 59, 7],
       [12, 2, 366, 22, 7]], dtype=object)

In [16]:
x_example, y_example = X_question[0], y_answer[0]
print("x:\n%s\n%s" % (" ".join([wordlist[x] for x in x_example]), x_example))
print("\ny:\n%s\n%s" % (" ".join([wordlist[x] for x in y_example]), y_example))

x:
in a program the code is this try { } finally { } whether program compiles & runs successfully ?
[5, 8, 184, 0, 72, 2, 41, 447, 136, 122, 878, 136, 122, 258, 184, 1111, 389, 362, 1228, 7]

y:
yes , try..finally can be possible without catch block
[140, 3, 1468, 13, 15, 161, 145, 541, 186]


In [17]:
import numpy as np

x = X_question
y = y_answer
max_length =  45
x_padded = []
y_padded = []


for row in x:
    if len(row) <= max_length:
        x_padded.append(row + [1719] * (max_length - len(row)))
    else :
        x_padded.append(row[:45])

for row in y:
    if len(row) <= max_length:
        y_padded.append(row + [1719] * (max_length - len(row))) 
    else: 
        y_padded.append(row[:45])


### Test Batch Pre-Processing

In [18]:
with open('Interviewtestdata_Question.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    testquestions = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(testquestions))

with open('Interviewtestdata_Answer.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    testanswers = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(testanswers)) 

with open('Interviewtestdata_label.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    testlabels = np.array(list(itertools.chain(*[x for x in reader]))).astype(np.int32)
    print(len(testlabels))

4
4
4


In [19]:
for i, sent in enumerate(testquestions):
    testquestions[i] = [w if w in word_to_index else unknown_token for w in sent]
    
for i, sent in enumerate(testanswers):
    testanswers[i] = [w if w in word_to_index else unknown_token for w in sent]

In [20]:
X_testquestion = np.asarray([[word_to_index[w] for w in sent] for sent in testquestions])
y_testanswer = np.asarray([[word_to_index[w] for w in sent] for sent in testanswers])

In [21]:
max_length = 45
x_testpadded = []
y_testpadded = []
p = X_testquestion
q = y_testanswer

for row in p:
    if len(row) <= max_length:
        x_testpadded.append(row + [1719] * (max_length - len(row))) 
    else :
        x_testpadded.append(row[:45])

for row in q:
    if len(row) <= max_length:
        y_testpadded.append(row + [1719] * (max_length - len(row)))
    else :
        y_testpadded.append(row[:45])

In [74]:
questionlength = []
answerlength = []
testquestionlength = []
testanswerlength = []

for i in X_question:
    questionlength.append(len(i) if len(i)<45 else 45)
for i in y_answer:
    answerlength.append(len(i) if len(i)<45 else 45)

    
for i in X_testquestion:
    testquestionlength.append(len(i) if len(i)<45 else 45)
for i in y_testanswer:
    testanswerlength.append(len(i) if len(i)<45 else 45)
    


In [81]:
tf.reset_default_graph()

from tensorflow.contrib import rnn 
hm_epochs = 10
batch_size = 4
state_size = 200
vocab_size = 1720

# Placeholders
context = tf.placeholder(tf.int32, [batch_size, None], name = "context") # [batch_size, num_steps]
context_len = tf.placeholder(tf.int32, [batch_size], name = "context_len")
utterance = tf.placeholder(tf.int32, [batch_size,None], name = "utterance")
utterance_len = tf.placeholder(tf.int32, [batch_size], name = "utterance_len")
targets = tf.placeholder(tf.int32, [batch_size], name = "targets")
target = tf.placeholder(tf.int32, [batch_size], name = "target")

def next_batch(step):
    p,q = x_padded[batch_size*step:batch_size*(step+1)], y_padded[batch_size*step:batch_size*(step+1)]
    r = np.array(questionlength[batch_size*step:batch_size*(step+1)])
    s = np.array(answerlength[batch_size*step:batch_size*(step+1)])
    t = labels[batch_size*step:batch_size*(step+1)]
    return p,q,r,s,t

def test_batch(step):
    a,b = x_testpadded[batch_size*step:batch_size*(step+1)], y_testpadded[batch_size*step:batch_size*(step+1)]
    c = np.array(testquestionlength[batch_size*step:batch_size*(step+1)])
    d = np.array(testanswerlength[batch_size*step:batch_size*(step+1)])
    e = testlabels[batch_size*step:batch_size*(step+1)]
    return a,b,c,d,e
        
def dual_encoder_model(context,context_len,utterance,utterance_len,targets):
    
    #Embedding the question and answer
    context_embedded = tf.nn.embedding_lookup(embeddings, context, name="embed_context")
    utterance_embedded = tf.nn.embedding_lookup(embeddings, utterance, name="embed_utterance")
    
    print(" This is the embeddings for both Q&A:", context_embedded, utterance_embedded)
    
    #initial_state = tf.tile(tf.get_variable('initial_state', [1, state_size],
                     #   initializer=tf.constant_initializer(0.0)),[batch_size,1])
    #this is thr RNN model 
    with tf.variable_scope("rnn") as vs:
        cell = tf.contrib.rnn.LSTMCell(state_size)
        
    #add a dropout layer to prevent overfitting
    cell = tf.contrib.rnn.DropoutWrapper(cell=cell, output_keep_prob = 0.5)
    #feed inputs to dynamic rnn for training    
    print("These are the inputs to RNN:", tf.concat([context_embedded, utterance_embedded],0))
    
    rnn_outputs, rnn_states = tf.nn.dynamic_rnn(
                                        cell,
                                        tf.concat([context_embedded, utterance_embedded],0),
                                        sequence_length=tf.concat([context_len, utterance_len],0),
                                        dtype=tf.float64)
    print("These are the outputs from RNN:", rnn_outputs, "This are the states outputs:", rnn_states)
    
    encoding_context, encoding_utterance = tf.split(rnn_states.h,2,0)
    
    print("This is the encoding context:", encoding_context)
    print("This is the encoding utterance:", encoding_utterance)
    
    with tf.variable_scope("prediction") as vs:
        M = tf.get_variable("M",
          shape=[state_size, state_size],
          initializer=tf.truncated_normal_initializer())

    # "Predict" a  response: c * M
    generated_response = tf.matmul(encoding_context, tf.cast(M, dtype= tf.float64), name = "encoding_context")
    print("This is the generated response:", generated_response)
    generated_response = tf.expand_dims(generated_response, 2, name = "generated_response")
    print("This is the generated response:", generated_response)
    encoding_utterance = tf.expand_dims(encoding_utterance, 2, name = "encoding_utterance")
    print("This is the encoding utterance:", encoding_utterance)
    # Dot product between generated response and actual response
    # (c * M) * r
    logit = tf.matmul(generated_response, encoding_utterance, True, name = "logits")
    print("This is the logits before squeeze:", logit)
    logits = tf.squeeze(logit, [2], name = "Squeeze")
    print("This is the logits after squeeze:", logits)

    # Apply sigmoid to convert logits to probabilities
    probs = tf.sigmoid(logits, name = "Sigmoid")   
    print("This is the probability after sigmoid:", probs)
    return tf.reshape(logits, [-1]), tf.reshape(probs, [-1])

def train_encoder(x):
    prediction, probs = dual_encoder_model(context,context_len,utterance,utterance_len,targets)
    # Calculate the binary cross-entropy loss
    print("This is the prediction size:", prediction)
    print("This is the target size:", targets)
    losses = tf.nn.sigmoid_cross_entropy_with_logits(logits = prediction, labels = tf.cast(targets, dtype= tf.float64))

    # Mean loss across the batch of examples
    cost = tf.reduce_mean(losses, name="cost")
    optimizer = tf.train.AdamOptimizer().minimize(cost)   
    saver = tf.train.Saver()
        
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        for epoch in range(hm_epochs):
            epoch_loss = 0 
            for step in range(int(len(x_padded)/batch_size)):
                ques, ans, queslength, anslength, labl = next_batch(step)
                _, c, logts, prob = sess.run([optimizer, cost, prediction, probs], feed_dict={context: ques, 
                                                              context_len: queslength,
                                                              utterance: ans, 
                                                              utterance_len: anslength, 
                                                              targets: labl})
                epoch_loss += c
                #print('step:', step, 'logits:', logts, 'probability:', prob)
            print('***************************** Epoch Completed *************************************')
            print('Epoch:', epoch, 'completed out of:',hm_epochs,'loss:',epoch_loss, 'probability:', prob)
        save_path = saver.save(sess, "./Interviewmodel")
        print("Model saved in file: %s" % save_path)  
        qe, an, quelength, anlength, lbl = test_batch(0)
        probas = sess.run(probs, feed_dict={context: qe, 
                                                              context_len: quelength,
                                                              utterance: an, 
                                                              utterance_len: anlength, 
                                                              target: lbl})
        print("Probs:", probas)

In [82]:
train_encoder(x)

 This is the embeddings for both Q&A: Tensor("embed_context:0", shape=(4, ?, 200), dtype=float64) Tensor("embed_utterance:0", shape=(4, ?, 200), dtype=float64)
These are the inputs to RNN: Tensor("concat:0", shape=(8, ?, 200), dtype=float64)
These are the outputs from RNN: Tensor("rnn_1/transpose:0", shape=(8, ?, 200), dtype=float64) This are the states outputs: LSTMStateTuple(c=<tf.Tensor 'rnn_1/while/Exit_2:0' shape=(?, 200) dtype=float64>, h=<tf.Tensor 'rnn_1/while/Exit_3:0' shape=(?, 200) dtype=float64>)
This is the encoding context: Tensor("split:0", shape=(?, 200), dtype=float64)
This is the encoding utterance: Tensor("split:1", shape=(?, 200), dtype=float64)
This is the generated response: Tensor("encoding_context:0", shape=(?, 200), dtype=float64)
This is the generated response: Tensor("generated_response:0", shape=(?, 200, 1), dtype=float64)
This is the encoding utterance: Tensor("encoding_utterance:0", shape=(?, 200, 1), dtype=float64)
This is the logits before squeeze: Tenso

In [83]:
with open('Interviewtestdata_Answer.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    testanswers = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(testanswers)) 
    
for i, sent in enumerate(testanswers):
    testanswers[i] = [w if w in word_to_index else unknown_token for w in sent]
    
y_testanswer = np.asarray([[word_to_index[w] for w in sent] for sent in testanswers])

max_length = 45

y_testpadded = []
for row in y_testanswer:
    if len(row) <= max_length:
        y_testpadded.append(row + [1719] * (max_length - len(row)))
    else :
        y_testpadded.append(row[:45])

answerlength = []        
for i in y_testanswer:
    answerlength.append(len(i) if len(i)<45 else 45)
    

4


In [84]:
for i in y_testpadded:
    print("x:\n%s\n\n%s" % (" ".join([wordlist[x] for x in i]),i))

x:
i UNKNOWN_TOKEN not UNKNOWN_TOKEN , i need an UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN

[125, 1719, 40, 1719, 3, 125, 98, 33, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719, 1719]
x:
i UNKNOWN_TOKEN not sure what are you UNKNOWN_TOKEN , a block ? UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN

In [85]:
#def test_batch(step):
    #a,b = x_testpadded[batch_size*step:batch_size*(step+1)], y_testpadded[batch_size*step:batch_size*(step+1)]
    #e = testlabels[batch_size*step:batch_size*(step+1)]
    #return a,b,e

sess=tf.Session()    
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('Interviewmodel.meta')
saver.restore(sess,tf.train.latest_checkpoint('./'))
sess.run(tf.global_variables_initializer())
graph = tf.get_default_graph()

context = graph.get_tensor_by_name("context:0") 
utterance = graph.get_tensor_by_name("utterance:0")
target = graph.get_tensor_by_name("target:0")
context_len = graph.get_tensor_by_name("context_len:0")
utterance_len = graph.get_tensor_by_name("utterance_len:0")

op_to_restore = graph.get_tensor_by_name("Sigmoid:0")
#qe,an,lbl = test_batch(0)
feed_dict={context: x_testpadded, utterance: y_testpadded, context_len: testquestionlength, utterance_len:testanswerlength, target: testlabels}
print(sess.run(op_to_restore,feed_dict))

[[ 0.55680219]
 [ 0.69608058]
 [ 0.44159931]
 [ 0.87232463]]
